In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
from random import randrange

from datetime import datetime, timedelta, date

import re

import pandas as pd

import os

In [50]:
dialogues_dir = 'dialogues'
my_name = 'Oleg Litvinov'
os.listdir(dialogues_dir)

['Oleg_Litvinov.csv',
 'Oleg_Litvinov.txt',
 'Влад Михайлович_Власенко.csv',
 'Влад_Михайлович_Власенко.txt',
 'Катя_Тимуровна.csv',
 'Полина.csv']

In [51]:
df = pd.read_csv(os.path.join(dialogues_dir, 'Катя_Тимуровна.csv'))
df.datetime = pd.to_datetime(df.datetime)
df.sort_values(['datetime'], inplace=True)
df.head()

,datetime,sender_id,sender_name,message
0,2017-03-20 22:34:59+00:00,47683,Oleg Litvinov,1. Берёшь крутой стикер\n2. Пишешь сообщение\n...
1,2017-03-20 22:35:10+00:00,47684,Oleg Litvinov,<MessageMediaDocument>
2,2017-03-20 22:35:14+00:00,47685,Oleg Litvinov,Привет
3,2017-03-20 22:36:27+00:00,47686,Катя Тимуровна,"Просто оказалось, что надо было дать телеграму..."
4,2017-03-20 22:36:48+00:00,47687,Oleg Litvinov,Что за щитовые смайлы?)


In [52]:
df.shape

(253051, 4)

In [6]:
df = df[~df.isna().any(axis=1)]
df.shape

(157716, 4)

In [7]:
# df[df.message.apply(lambda x: re.search(r'(MessageMediaPhoto)', x) is not None)].message.apply(lambda x: re.sub(r'(<\w*Media\w*> )', '', x))

In [8]:
df.message = df.message.apply(lambda x: re.sub(r'(<\w*Media\w*> )', '', x))

df = df[df.message.apply(lambda x: len(x)) > 0]

df.shape

(143078, 4)

In [9]:
df.head()

,datetime,sender_id,sender_name,message
0,2016-10-17 19:46:36+00:00,274,Влад Михайлович Власенко,"Ок, ватс см удалил"
3,2016-10-17 19:54:21+00:00,280,Влад Михайлович Власенко,Лол)
4,2016-10-17 19:54:51+00:00,281,Oleg Litvinov,Норм)\nЯ спать пойду)\nВ 7 вождение)
5,2016-10-17 19:54:53+00:00,282,Oleg Litvinov,Спок)
6,2016-10-17 19:54:57+00:00,283,Влад Михайлович Власенко,Теплых


In [10]:
df['from_prev_msg'] = df.datetime - df.datetime.shift(1)

In [11]:
df.from_prev_msg = df.from_prev_msg.astype('timedelta64[h]')
df

,datetime,sender_id,sender_name,message,from_prev_msg
0,2016-10-17 19:46:36+00:00,274,Влад Михайлович Власенко,"Ок, ватс см удалил",NaN
3,2016-10-17 19:54:21+00:00,280,Влад Михайлович Власенко,Лол),0.0
4,2016-10-17 19:54:51+00:00,281,Oleg Litvinov,Норм)\nЯ спать пойду)\nВ 7 вождение),0.0
5,2016-10-17 19:54:53+00:00,282,Oleg Litvinov,Спок),0.0
6,2016-10-17 19:54:57+00:00,283,Влад Михайлович Власенко,Теплых,0.0
...,...,...,...,...,...
157815,2021-04-02 17:30:33+00:00,695478,Влад Михайлович Власенко,И по времени тож,0.0
157816,2021-04-02 17:36:07+00:00,695479,Влад Михайлович Власенко,Но вроде как туристам нельзя или карантин,0.0
157817,2021-04-02 17:57:00+00:00,695481,Oleg Litvinov,лел,0.0
157818,2021-04-03 08:34:44+00:00,695485,Oleg Litvinov,что с нами не так,14.0


In [12]:
df['new_topic'] = df['from_prev_msg'] > 4
df.head()

,datetime,sender_id,sender_name,message,from_prev_msg,new_topic
0,2016-10-17 19:46:36+00:00,274,Влад Михайлович Власенко,"Ок, ватс см удалил",NaN,False
3,2016-10-17 19:54:21+00:00,280,Влад Михайлович Власенко,Лол),0.0,False
4,2016-10-17 19:54:51+00:00,281,Oleg Litvinov,Норм)\nЯ спать пойду)\nВ 7 вождение),0.0,False
5,2016-10-17 19:54:53+00:00,282,Oleg Litvinov,Спок),0.0,False
6,2016-10-17 19:54:57+00:00,283,Влад Михайлович Власенко,Теплых,0.0,False


In [13]:
df.reset_index(drop=True, inplace=True)
df.head()

,datetime,sender_id,sender_name,message,from_prev_msg,new_topic
0,2016-10-17 19:46:36+00:00,274,Влад Михайлович Власенко,"Ок, ватс см удалил",NaN,False
1,2016-10-17 19:54:21+00:00,280,Влад Михайлович Власенко,Лол),0.0,False
2,2016-10-17 19:54:51+00:00,281,Oleg Litvinov,Норм)\nЯ спать пойду)\nВ 7 вождение),0.0,False
3,2016-10-17 19:54:53+00:00,282,Oleg Litvinov,Спок),0.0,False
4,2016-10-17 19:54:57+00:00,283,Влад Михайлович Власенко,Теплых,0.0,False


In [14]:
df.sender_name.value_counts()

Влад Михайлович Власенко    74990
Oleg Litvinov               68088
Name: sender_name, dtype: int64

In [15]:
df['is_answer'] = df.sender_name != df.sender_name.shift(1)

In [16]:
df.head()

,datetime,sender_id,sender_name,message,from_prev_msg,new_topic,is_answer
0,2016-10-17 19:46:36+00:00,274,Влад Михайлович Власенко,"Ок, ватс см удалил",NaN,False,True
1,2016-10-17 19:54:21+00:00,280,Влад Михайлович Власенко,Лол),0.0,False,False
2,2016-10-17 19:54:51+00:00,281,Oleg Litvinov,Норм)\nЯ спать пойду)\nВ 7 вождение),0.0,False,True
3,2016-10-17 19:54:53+00:00,282,Oleg Litvinov,Спок),0.0,False,False
4,2016-10-17 19:54:57+00:00,283,Влад Михайлович Власенко,Теплых,0.0,False,True


In [37]:
from tqdm import tqdm

In [45]:
def generate_train_file(df):
    final_string = []
    for index, series in tqdm(df.iterrows(), total=len(df)):
#         print(index)
        if (series['sender_name'] == my_name) and (series['is_answer'] == True) and (index > 10):
            story_len = randrange(1, min(30, index))
#             print('story_len', story_len)
            history_df = df.loc[index-story_len:index+1, ['sender_name', 'message']]
            output_string = "<s>" + "\n".join((history_df['sender_name'] + ": " + history_df['message']).values)
#             print(output_string)
            final_string.append(output_string)
    final_string = "\n".join(final_string)
    return final_string

In [32]:
train_end = date(2020, 1, 1)
val_end = date(2021, 1, 1)
test_end = df.datetime.max().date() - timedelta(weeks=1)

In [33]:
train_df = df[df.datetime.dt.date < train_end]
val_df = df[(df.datetime.dt.date < val_end) & (df.datetime.dt.date >= train_end)]
test_df = df[(df.datetime.dt.date < test_end) & (df.datetime.dt.date >= val_end)]

In [35]:
train_df.shape, val_df.shape, test_df.shape

((119384, 7), (21822, 7), (1690, 7))

In [46]:
test_file = generate_train_file(test_df)

100%|████████████████████████████████████████████████████████████████████████████| 1690/1690 [00:00<00:00, 2381.65it/s]


In [39]:
train_file = generate_train_file(train_df)
val_df = generate_train_file(val_df)

100%|███████████████████████████████████████████████████████████████████████████| 21822/21822 [00:41<00:00, 521.21it/s]


In [44]:
with open("train.txt", "w", encoding="utf-8") as text_file:
    text_file.write(train_file)

In [43]:
with open("valid.txt", "w", encoding="utf-8") as text_file:
    text_file.write(val_df)

In [47]:
with open("test.txt", "w", encoding="utf-8") as text_file:
    text_file.write(test_file)